In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter
import sqlalchemy
import hashlib
import glob

### Electoral Districts (2003 representation order)

In [405]:
%%file db/electoral_district.sql

DROP TABLE IF EXISTS electoral_district;

CREATE TABLE electoral_district (
    nid      TEXT PRIMARY KEY, --National identifier. A universally unique identifier
    fednum   INTEGER,          --A unique number that identifies a FED
    enname   TEXT,             --The official English name for the FED
    frname   TEXT,             --The official French name for the FED
    provcode TEXT,             --Province or territory covered by the dataset
    creadt   TEXT,             --The date when the object was originally created 
    revdt    TEXT,             --The date when the object was last revised
    reporder TEXT,             --The year of the representation order the FED is related to
    decpopcnt INTEGER,         --Decennial census population count of the FED
    quipopcnt INTEGER          --Quinquennale census population count of the FED
);

Writing db/electoral_district.sql


In [492]:
!sqlite3 db/election.db < db/electoral_district.sql

In [24]:
%%bash
wget \
  http://ftp2.cits.rncan.gc.ca/pub/geobase/official/fed_cf/shp_eng/fed_cf_CA_1_1_shp_en.zip \
  -O data/fed_cf_CA_1_1_shp_en.zip
    
unzip -u \
  data/fed_cf_CA_1_1_shp_en.zip \
  -d data
  
ogr2ogr -f GeoJSON data/FED_CA_1_1_en.geojson data/FED_CA_1_1_en.shp

Archive:  data/fed_cf_CA_1_1_shp_en.zip


--2015-08-29 15:19:02--  http://ftp2.cits.rncan.gc.ca/pub/geobase/official/fed_cf/shp_eng/fed_cf_CA_1_1_shp_en.zip
Resolving ftp2.cits.rncan.gc.ca... 192.67.45.79
Connecting to ftp2.cits.rncan.gc.ca|192.67.45.79|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9633737 (9.2M) [application/zip]
Saving to: 'data/fed_cf_CA_1_1_shp_en.zip'

     0K .......... .......... .......... .......... ..........  0%  193M 0s
    50K .......... .......... .......... .......... ..........  1%  191M 0s
   100K .......... .......... .......... .......... ..........  1%  418K 7s
   150K .......... .......... .......... .......... ..........  2%  176M 6s
   200K .......... .......... .......... .......... ..........  2%  256M 4s
   250K .......... .......... .......... .......... ..........  3%  755K 6s
   300K .......... .......... .......... .......... ..........  3%  158M 5s
   350K .......... .......... .......... .......... ..........  4%  856K 6s
   400K .......... .......... 

In [26]:
FED_data = json.load( open('data/FED_CA_1_1_en.geojson') )

In [407]:
records = []
for FED in FED_data['features']:
    records.append(FED['properties'])

In [2]:
engine = sqlalchemy.create_engine("sqlite:///db/election.db", encoding='latin-1')

In [493]:
districts = pd.DataFrame(records, columns=['NID',
                                           'FEDNUM',
                                           'ENNAME',
                                           'FRNAME',
                                           'PROVCODE',
                                           'CREADT',
                                           'REVDT',
                                           'REPORDER',
                                           'DECPOPCNT',
                                           'QUIPOPCNT'])
districts.columns = ['nid','fednum','enname','frname','provcode','creadt','revdt','reporder','decpopcnt','quipopcnt']
districts.drop_duplicates().to_sql('electoral_district', engine, if_exists='append', index=False)

### Election results (2011)

In [90]:
%%bash
wget \
  http://www.elections.ca/scripts/OVR2011/34/data_donnees/pollresults_resultatsbureau_canada.zip \
  -O data/pollresults_resultatsbureau_canada.zip
    
unzip -u \
  data/pollresults_resultatsbureau_canada.zip \
  -d data

Archive:  data/pollresults_resultatsbureau_canada.zip


--2015-08-29 17:51:45--  http://www.elections.ca/scripts/OVR2011/34/data_donnees/pollresults_resultatsbureau_canada.zip
Resolving www.elections.ca... 184.150.235.237
Connecting to www.elections.ca|184.150.235.237|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3085377 (2.9M) [application/x-zip-compressed]
Saving to: 'data/pollresults_resultatsbureau_canada.zip'

     0K .......... .......... .......... .......... ..........  1%  131M 0s
    50K .......... .......... .......... .......... ..........  3%  237M 0s
   100K .......... .......... .......... .......... ..........  4% 56.3K 17s
   150K .......... .......... .......... .......... ..........  6%  192M 13s
   200K .......... .......... .......... .......... ..........  8%  200M 10s
   250K .......... .......... .......... .......... ..........  9% 34.8K 21s
   300K .......... .......... .......... .......... .......... 11%  239M 18s
   350K .......... .......... .......... .......... .......... 13% 25.0K 

In [457]:
%%file db/candidate.sql

DROP TABLE IF EXISTS candidate;

CREATE TABLE candidate (
    id                    TEXT NOT NULL PRIMARY KEY,
    family_name           TEXT NOT NULL,
    middle_name           TEXT,
    first_name            TEXT NOT NULL,
    political_affiliation TEXT NOT NULL,
    incumbent             BOOLEAN NOT NULL,
    elected               BOOLEAN NOT NULL
);

Overwriting db/candidate.sql


In [458]:
%%file db/station.sql

DROP TABLE IF EXISTS station;

CREATE TABLE station (
    id           TEXT PRIMARY KEY,
    name         TEXT,
    number       TEXT,
    void         BOOLEAN,
    no_poll      BOOLEAN,
    electors     INTEGER,
    merge_with   TEXT,
    rejected     INTEGER
);

Overwriting db/station.sql


In [459]:
%%file db/poll.sql

DROP TABLE IF EXISTS poll;

CREATE TABLE poll (
    id           TEXT PRIMARY KEY,
    fednum       INTEGER,
    station_id   TEXT,
    votes        INTEGER,
    candidate_id TEXT,
    FOREIGN KEY(fednum)       REFERENCES electoral_district(fednum),
    FOREIGN KEY(station_id)   REFERENCES station(id),
    FOREIGN KEY(candidate_id) REFERENCES candidate(id)
);

Overwriting db/poll.sql


In [469]:
%%bash
sqlite3 db/election.db < db/candidate.sql
sqlite3 db/election.db < db/station.sql
sqlite3 db/election.db < db/poll.sql

In [470]:
poll_files = glob.glob("data/pollresults_resultatsbureau*.csv")

In [471]:
for poll_file in poll_files:

    #read in the poll file, and rename the columns so that pandas has an easier time
    district_results = pd.read_csv(poll_file,
                                   encoding='latin-1',
                                   skiprows=1,
                                   names=[u'Electoral District Number',
                                          u'Electoral District Name_English',
                                          u'Electoral District Name_French',
                                          u'Polling Station Number',
                                          u'Polling Station Name',
                                          u'Void Poll Indicator',
                                          u'No Poll Held Indicator',
                                          u'Merge With',
                                          u'Rejected Ballots for Polling Station',
                                          u'Electors for Polling Station',
                                          u"Candidate's Family Name",
                                          u"Candidate's Middle Name",
                                          u"Candidate's First Name",
                                          u'Political Affiliation Name_English',
                                          u'Political Affiliation Name_French',
                                          u'Incumbent Indicator',
                                          u'Elected Candidate Indicator',
                                          u'Candidate Poll Votes Count'])

    #candidate_id definition
    district_results["candidate_id"] = \
      district_results["Candidate's First Name"] + \
      district_results["Candidate's Middle Name"].map(unicode) + \
      district_results["Candidate's Family Name"] + \
      district_results["Political Affiliation Name_English"]
    
    #station_id definition
    district_results["station_id"] = \
      district_results["Electoral District Number"].map(unicode) + \
      district_results["Polling Station Name"] + \
      district_results["Polling Station Number"]
        
    #poll_id definition
    district_results["poll_id"] = \
      district_results["station_id"] + \
      district_results["candidate_id"]
    
    #turn the concated ids into hash values
    def hash_id(x):
        return hashlib.md5(x.encode('utf-8')).hexdigest()
    
    district_results['poll_id']      = district_results['poll_id'].map(hash_id)
    district_results['station_id']   = district_results['station_id'].map(hash_id)
    district_results['candidate_id'] = district_results['candidate_id'].map(hash_id)
    
    #normalized the candidate table
    candidates = pd.DataFrame(district_results, columns=["candidate_id",
                                                         "Candidate's First Name",
                                                         "Candidate's Middle Name",
                                                         "Candidate's Family Name",
                                                         "Political Affiliation Name_English",
                                                         "Incumbent Indicator",
                                                         "Elected Candidate Indicator"])
    candidates.columns = ["id", "first_name", "middle_name", "family_name", "political_affiliation", "incumbent", "elected"]
    candidates['incumbent'] = candidates['incumbent'].apply(lambda x: True if x=='Y' else False)
    candidates['elected']   = candidates['elected'].apply(lambda x: True if x=='Y' else False)
    
    #insert the candidate rows
    candidates.drop_duplicates().to_sql('candidate', engine, if_exists='append', index=False)

    #normalize station table
    stations = pd.DataFrame(district_results, columns = ["station_id",
                                                         "Polling Station Name",
                                                         "Polling Station Number",
                                                         "Void Poll Indicator",
                                                         "No Poll Held Indicator",
                                                         "Electors for Polling Station",
                                                         "Merge With",
                                                         "Rejected Ballots for Polling Station"])
    stations.columns    = ["id","name","number","void","no_poll","electors","merge_with","rejected"]
    stations['void']    = stations['void'].apply(lambda x: True if x=='Y' else False)
    stations['no_poll'] = stations['no_poll'].apply(lambda x: True if x=='Y' else False)
    
    #insert station rows
    stations.drop_duplicates().to_sql('station', engine, if_exists='append', index=False)

    #normalize poll table
    polls = pd.DataFrame(district_results, columns=["poll_id",
                                                    "Electoral District Number",
                                                    "station_id",
                                                    "Candidate Poll Votes Count",
                                                    "candidate_id"
                                                   ])
    polls.columns = ['id','fednum','station_id','votes','candidate_id']
    
    #insert poll rows
    polls.to_sql('poll', engine, if_exists='append', index=False)


### First past the post

In [121]:
fptp = pd.read_sql("""
               SELECT 
                 poll.fednum,
                 electoral_district.enname,
                 electoral_district.provcode,
                 candidate.family_name,
                 candidate.political_affiliation,
                 SUM(poll.votes) as votes
               FROM poll, candidate, electoral_district
               WHERE poll.candidate_id=candidate.id
                 AND poll.fednum=electoral_district.fednum
               GROUP BY poll.candidate_id, poll.fednum
               ORDER BY poll.fednum, votes DESC
            """, engine).groupby('fednum')

In [122]:
fptp.first().head()

,enname,provcode,family_name,political_affiliation,votes
fednum,,,,,
10001,Avalon,NL,Andrews,Liberal,16008
10002,Bonavista--Gander--Grand Falls--Windsor,NL,Simms,Liberal,17977
10003,Humber--St. Barbe--Baie Verte,NL,Byrne,Liberal,17119
10004,Labrador,NL,Penashue,Conservative,4256
10005,Random--Burin--St. George's,NL,Foote,Liberal,12914


In [123]:
Counter(fptp.first().political_affiliation.values)

Counter({u'Bloc Qu\xe9b\xe9cois': 4,
         u'Conservative': 166,
         u'Green Party': 1,
         u'Liberal': 34,
         u'NDP-New Democratic Party': 103})

In [124]:
fptp.first().votes.sum()

7495961

In [127]:
fptp.votes.sum().values.sum()

14873962

In [128]:
7495961./14873962

0.5039653187227452

### Instant runoff voting

http://www.ekospolitics.com/wp-content/uploads/full_report_april_29_2011.pdf

| Party                       |Cons|Libs | NDP |Green|Bloc|Other| Nope|
|-----------------------------|----|-----|-----|-----|----|-----|-----|
|Conservative Party of Canada |  0%| 16% | 21% | 11% | 1% |  4% | 47% |
|Liberal Party of Canada      | 13%|  0% | 54% | 12% | 3% |  1% | 17% |
|NDP                          | 14%| 38% |  0% | 19% |11% |  1% | 17% |
|Green Party                  | 11%| 17% | 40% |  0% | 3% |  1% | 27% |
|Bloc Quebecois               |  7%| 13% | 49% |  8% | 0% |  1% | 22% |
|Undecided                    | 11%|  9% | 16% | 12% | 9% |  0% | 43% |

In [6]:
second_choices = {
    'Conservative':{
        'Liberal':0.16,
        'NDP-New Democratic Party':0.21,
        'Green Party':0.11,
        u'Bloc Qu\xe9b\xe9cois':0.01,
        'Other':0.04,
        'None':0.47
    },
    'Liberal':{
        'Conservative':0.13,
        'NDP-New Democratic Party':0.54,
        'Green Party':0.12,
        u'Bloc Qu\xe9b\xe9cois':0.03,
        'Other':0.01,
        'None':0.17
    },
    'NDP-New Democratic Party':{
        'Conservative':0.14,
        'Liberal':0.38,
        'Green Party':0.19,
        u'Bloc Qu\xe9b\xe9cois':0.11,
        'Other':0.01,
        'None':0.17    
    },
    'Green Party':{
        'Conservative':0.11,
        'Liberal':0.17,
        'NDP-New Democratic Party':0.40,
        u'Bloc Qu\xe9b\xe9cois':0.03,
        'Other':0.01,
        'None':0.28 
    },
    u'Bloc Qu\xe9b\xe9cois':{
        'Conservative':0.07,
        'Liberal':0.13,
        'NDP-New Democratic Party':0.49,
        'Green Party':0.08,
        'Other':0.01,
        'None':0.22 
    },
    'default':{
        'Conservative':0.10,
        'Liberal':0.17,
        'NDP-New Democratic Party':0.32,
        'Green Party':0.11,
        u'Bloc Qu\xe9b\xe9cois':0.04,
        'Other':0.02,
        'None':0.24
    }
}

In [7]:
def runoff_round(candidates, choice_matrix):
    if len(candidates) == 0:
        return []
    
    #initialize the change-log to empty set
    change_log = []
    
    #sort the candidates by votes, and find the one with the fewest votes
    sorted_by_votes  = sorted([candidate for candidate in candidates if candidate['votes']>0], key=lambda x: x['votes'])
    lowest_candidate = sorted_by_votes[0]
    
    #the candidate with the lowest votes will have their votes redistributed, so subtract from their total
    change_log.append( {'id':lowest_candidate['id'], 'votes':-1*lowest_candidate['votes']} )
    
    choice_vector = choice_matrix.get(lowest_candidate['political_affiliation'], choice_matrix['default'])
    
    #renormalize choice vector based on the remaining parties
    remaining_parties = set([candidate['political_affiliation'] for candidate in sorted_by_votes]) - set([lowest_candidate['political_affiliation']])
    remaining_choices = set(choice_vector.keys())
    parties = remaining_parties.intersection(remaining_choices)

    party_weights = np.array([choice_vector[x] for x in parties])
    party_weights = party_weights/np.sum(party_weights)

    redistribution = Counter( np.random.choice(list(parties), p=party_weights, size=lowest_candidate['votes'] ) )
    
    #make a index of party->candidate_id
    index = dict([(candidate['political_affiliation'], candidate['id']) for candidate in candidates] )
    
    for party, votes in redistribution.items():
        id = index[party]
        change_log.append({'id':id, 'votes':votes})

    return change_log
  

def apply_changes(candidates, change_log):

    candidate_index = dict([(candidate['id'], candidate.copy()) for candidate in candidates] )
    
    for round in change_log:
        for change in round:
            candidate_index[change['id']]['votes'] += change['votes']
    
    return candidate_index.values()


def instant_runoff( candidates, choice_matrix ):
    
    def elected(_candidates, _threshold):
        for candidate in _candidates:
            if candidate['votes'] >= _threshold:
                return candidate
        return None
    
    max_rounds = len(candidates) - 1
    threshold = 1 + sum([candidate['votes'] for candidate in candidates])/2.
    
    rounds = []
    for i in range(max_rounds):
        
        round_candidates = apply_changes(candidates, rounds)
        if elected(round_candidates, threshold):
            break
        
        rounds.append( runoff_round(round_candidates, choice_matrix) )
        
    _elected = elected(apply_changes(candidates, rounds), threshold)
        
    return {'candidates':candidates,
            'threshold':int(threshold),
            'elected':_elected['id'],
            'elected_party':_elected['political_affiliation'],
            'elected_votes':_elected['votes'],
            'rounds':rounds}

In [8]:
def simulate_instant_runoff():
    
    results = pd.read_sql("""
               SELECT 
                 poll.fednum,
                 candidate.id,
                 candidate.first_name || " " || candidate.family_name as name,
                 candidate.political_affiliation,
                 SUM(poll.votes) as votes
               FROM poll, candidate, electoral_district
               WHERE poll.candidate_id=candidate.id
                 AND poll.fednum=electoral_district.fednum
               GROUP BY poll.candidate_id, poll.fednum
               ORDER BY poll.fednum, votes DESC
            """, engine).groupby('fednum')
    
    simulated_election = {}
    
    for result in results:
        fednum, candidates = result

        candidates = candidates.to_dict(orient='records')
        simulated_election[str(fednum)] = instant_runoff(candidates, second_choices)
        
    return simulated_election

In [9]:
%%time
simulation = simulate_instant_runoff()

CPU times: user 2.52 s, sys: 183 ms, total: 2.71 s
Wall time: 2.77 s


In [15]:
Counter([x['elected_party'] for x in simulation.values()])

Counter({u'Conservative': 132,
         u'Green Party': 1,
         u'Liberal': 51,
         u'NDP-New Democratic Party': 124})

In [664]:
sum([x['elected_votes'] for x in simulation.values()])

8447063

In [10]:
8447063./14873962

0.5679094110903335

In [14]:
json.dump(simulation, open('irv_simulation.json','w'))